Загружаем библиотеки, которые на понадобятся 

In [1]:
import pandas as pd
import xgboost as xgb

Загружаем данные

In [2]:
transactions_train=pd.read_csv('data/transactions_train.csv')
train_target=pd.read_csv('data/train_target.csv')

Считаем агригационные признаки(сумма, стандартное отклонение, медианное значение, среднее арифметическое, минимум, максимум, квантили и количество транкзаций) для каждого из клиентов 

In [3]:
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','median','std','min','max','quantile','count']).reset_index()

Будем считать для каждого пользователя, агригационные признаки для каждой группы транкзаций

In [4]:
sum_df=transactions_train.groupby(['client_id','small_group'])['amount_rur'].sum()
mean_df=transactions_train.groupby(['client_id','small_group'])['amount_rur'].mean()
median_df=transactions_train.groupby(['client_id','small_group'])['amount_rur'].median()
count_df=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
quantile_df=transactions_train.groupby(['client_id','small_group'])['amount_rur'].quantile()
max_df=transactions_train.groupby(['client_id','small_group'])['amount_rur'].max()
min_df=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
std_df=transactions_train.groupby(['client_id','small_group'])['amount_rur'].std()

In [9]:
sum_df=sum_df.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
mean_df=mean_df.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
median_df=median_df.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
count_df=count_df.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
quantile_df=quantile_df.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
max_df=max_df.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
min_df=min_df.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
std_df=std_df.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')

In [10]:
sum_df=sum_df.fillna(0)
mean_df=mean_df.fillna(0)
median_df=median_df.fillna(0)
count_df=count_df.fillna(0)
quantile_df=quantile_df.fillna(0)
max_df=max_df.fillna(0)
min_df=min_df.fillna(0)
std_df=std_df.fillna(0)

In [11]:
sum_df.columns=['small_group_sum'+str(i) for i in sum_df.columns]
mean_df.columns=['small_group_mean'+str(i) for i in mean_df.columns]
median_df.columns=['small_group_median'+str(i) for i in median_df.columns]
count_df.columns=['small_group_count'+str(i) for i in count_df.columns]
quantile_df.columns=['small_group_quantile'+str(i) for i in quantile_df.columns]
max_df.columns=['small_group_max'+str(i) for i in max_df.columns]
min_df.columns=['small_group_min'+str(i) for i in min_df.columns]
std_df.columns=['small_group_std'+str(i) for i in std_df.columns]

удаляем ненужные группы, которые не используются

In [12]:
del median_df['small_group_median199']
del median_df['small_group_median203']
del sum_df['small_group_sum199']
del sum_df['small_group_sum203']
del mean_df['small_group_mean199']
del mean_df['small_group_mean203']
del count_df['small_group_count199']
del count_df['small_group_count203']
del quantile_df['small_group_quantile199']
del quantile_df['small_group_quantile203']
del max_df['small_group_max199']
del max_df['small_group_max203']
del min_df['small_group_min199']
del min_df['small_group_min203']
del std_df['small_group_std199']
del std_df['small_group_std203']

Все объединяем

In [13]:
train=pd.merge(train_target,agg_features,on='client_id')

In [14]:
train=pd.merge(train,sum_df.reset_index(),on='client_id')
train=pd.merge(train,mean_df.reset_index(),on='client_id')
train=pd.merge(train,median_df.reset_index(),on='client_id')
train=pd.merge(train,count_df.reset_index(),on='client_id')
train=pd.merge(train,quantile_df.reset_index(),on='client_id')
train=pd.merge(train,max_df.reset_index(),on='client_id')
train=pd.merge(train,min_df.reset_index(),on='client_id')
train=pd.merge(train,std_df.reset_index(),on='client_id')

Смотрим на данные, которые получили

In [15]:
train

,client_id,bins,sum,mean,median,std,min,max,quantile,count,...,small_group_std198,small_group_std200,small_group_std202,date_sum,date_max,date_min,date_count,date_median,date_quantile,date_std
0,24662,2,30254.011,34.774725,21.6995,72.037354,0.074,1227.314,21.6995,870,...,0.0,0.0,0.0,61.617130,45.008124,18.473047,1.771894,27.398067,27.398067,31.509236
1,1046,0,42548.570,52.015367,20.1500,106.540962,0.550,1210.506,20.1500,818,...,0.0,0.0,0.0,90.721898,67.226719,29.285096,1.744136,40.187048,40.187048,43.446176
2,34089,2,26842.816,34.325852,16.9020,59.927450,0.043,782.641,16.9020,782,...,0.0,0.0,0.0,53.578475,44.456858,23.556898,1.560878,31.439901,31.439901,32.341938
3,34848,1,15773.126,16.160990,12.0730,14.224936,0.043,109.590,12.0730,976,...,0.0,0.0,0.0,36.427543,23.198247,9.058065,2.254042,14.480434,14.480434,11.167202
4,47076,3,12488.375,15.929050,8.7065,35.473591,0.432,541.165,8.7065,784,...,0.0,0.0,0.0,26.017448,20.898654,12.470665,1.633333,15.882343,15.882343,11.413421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,14303,1,285001.426,345.456274,104.3690,1009.576868,1.786,9919.347,104.3690,825,...,0.0,0.0,0.0,612.906292,529.232630,206.736054,1.774194,278.902397,278.902397,392.931823
29996,22301,2,34950.321,49.715962,25.6450,116.602464,0.043,2136.877,25.6450,703,...,0.0,0.0,0.0,84.830876,67.390871,40.309301,1.706311,50.203818,50.203818,34.427388
29997,25731,0,54074.454,49.338005,22.8215,80.490725,0.181,884.970,22.8215,1096,...,0.0,0.0,0.0,100.697307,73.214089,25.421432,2.040968,41.055199,41.055199,43.901044
29998,16820,3,32494.435,42.643615,18.3190,136.489534,0.388,2959.186,18.3190,762,...,0.0,0.0,0.0,66.586957,54.141074,23.060982,1.561475,34.086946,34.086946,46.980943


Загружаем тестовые данные и тоже самое проделовываем с ними

In [16]:
transactions_test=pd.read_csv('data/transactions_test.csv')
test_id=pd.read_csv('data/test.csv')

In [17]:
agg_features_test=transactions_test.groupby('client_id')['amount_rur'].agg(['sum','mean','median','std','min','max','quantile','count']).reset_index()

In [21]:
sum_df_test=transactions_test.groupby(['client_id','small_group'])['amount_rur'].sum()
mean_df_test=transactions_test.groupby(['client_id','small_group'])['amount_rur'].mean()
median_df_test=transactions_test.groupby(['client_id','small_group'])['amount_rur'].median()
count_df_test=transactions_test.groupby(['client_id','small_group'])['amount_rur'].count()
quantile_df_test=transactions_test.groupby(['client_id','small_group'])['amount_rur'].quantile()
max_df_test=transactions_test.groupby(['client_id','small_group'])['amount_rur'].max()
min_df_test=transactions_test.groupby(['client_id','small_group'])['amount_rur'].min()
std_df_test=transactions_test.groupby(['client_id','small_group'])['amount_rur'].std()

In [23]:
sum_df_test=sum_df_test.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
mean_df_test=mean_df_test.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
median_df_test=median_df_test.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
count_df_test=count_df_test.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
quantile_df_test=quantile_df_test.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
max_df_test=max_df_test.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
min_df_test=min_df_test.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')
std_df_test=std_df_test.reset_index().pivot(index='client_id',columns='small_group',values='amount_rur')

In [24]:
sum_df_test=sum_df_test.fillna(0)
mean_df_test=mean_df_test.fillna(0)
median_df_test=median_df_test.fillna(0)
count_df_test=count_df_test.fillna(0)
quantile_df_test=quantile_df_test.fillna(0)
max_df_test=max_df_test.fillna(0)
min_df_test=min_df_test.fillna(0)
std_df_test=std_df_test.fillna(0)

In [25]:
sum_df_test.columns=['small_group_sum'+str(i) for i in sum_df_test.columns]
mean_df_test.columns=['small_group_mean'+str(i) for i in mean_df_test.columns]
median_df_test.columns=['small_group_median'+str(i) for i in median_df_test.columns]
count_df_test.columns=['small_group_count'+str(i) for i in count_df_test.columns]
quantile_df_test.columns=['small_group_quantile'+str(i) for i in quantile_df_test.columns]
max_df_test.columns=['small_group_max'+str(i) for i in max_df_test.columns]
min_df_test.columns=['small_group_min'+str(i) for i in min_df_test.columns]
std_df_test.columns=['small_group_std'+str(i) for i in std_df_test.columns]

In [27]:
test=pd.merge(test_id,agg_features_test,on='client_id')

In [28]:
test=pd.merge(test,sum_df_test.reset_index(),on='client_id')
test=pd.merge(test,mean_df_test.reset_index(),on='client_id')
test=pd.merge(test,median_df_test.reset_index(),on='client_id')
test=pd.merge(test,count_df_test.reset_index(),on='client_id')
test=pd.merge(test,quantile_df_test.reset_index(),on='client_id')
test=pd.merge(test,max_df_test.reset_index(),on='client_id')
test=pd.merge(test,min_df_test.reset_index(),on='client_id')
test=pd.merge(test,std_df_test.reset_index(),on='client_id')

In [29]:
del test['small_group_median194']
del test['small_group_median201']
del test['small_group_sum194']
del test['small_group_sum201']
del test['small_group_mean194']
del test['small_group_mean201']
del test['small_group_count194']
del test['small_group_count201']
del test['small_group_quantile194']
del test['small_group_quantile201']
del test['small_group_max194']
del test['small_group_max201']
del test['small_group_min194']
del test['small_group_min201']
del test['small_group_std194']
del test['small_group_std201']

Смотрим на данные, которые получили

In [31]:
test

,client_id,sum,mean,median,std,min,max,quantile,count,small_group_sum0,...,small_group_std198,small_group_std200,small_group_std202,date_sum,date_max,date_min,date_count,date_median,date_quantile,date_std
0,28571,30507.083,42.488974,28.0745,43.659666,0.078,306.882,28.0745,718,0.000,...,0.0,0.0,0.0,66.755105,54.360790,31.043427,1.571116,40.413229,40.413229,35.508067
1,27046,39378.016,38.988135,17.0350,93.600961,0.043,1469.007,17.0350,1010,468.444,...,0.0,0.0,0.0,89.090534,65.219238,22.582611,2.285068,34.603184,34.603184,35.708745
2,13240,50211.127,53.302683,16.4635,198.583630,1.078,3902.918,16.4635,942,0.000,...,0.0,0.0,0.0,106.379506,86.914513,30.736400,1.995763,46.196806,46.196806,56.634558
3,19974,45371.491,53.252924,18.8575,273.597147,0.432,5865.551,18.8575,852,1417.209,...,0.0,0.0,0.0,89.313959,75.377437,26.829669,1.677165,36.144737,36.144737,59.842002
4,10505,90883.628,81.950972,51.6070,119.408720,0.043,1921.341,51.6070,1109,0.000,...,0.0,0.0,0.0,157.238111,122.788246,47.399348,1.918685,72.369275,72.369275,75.015310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2565,35397.209,35.397209,12.5435,139.556590,0.051,3202.483,12.5435,1000,0.000,...,0.0,0.0,0.0,71.221748,53.832948,12.002354,2.012072,22.656778,22.656778,41.760117
19996,31255,17369.096,16.928943,6.6810,36.466146,0.129,646.914,6.6810,1026,11.220,...,0.0,0.0,0.0,31.869901,24.319690,9.368400,1.882569,13.722752,13.722752,15.714999
19997,31539,27405.309,23.913882,14.3180,38.592933,0.043,674.429,14.3180,1146,0.000,...,0.0,0.0,0.0,45.000507,34.107082,15.967399,1.881773,22.535759,22.535759,18.411073
19998,4288,24117.923,29.628898,13.7380,66.501257,0.229,1007.405,13.7380,814,0.000,...,0.0,0.0,0.0,56.614843,44.511657,17.252366,1.910798,24.390448,24.390448,28.951342


Распределим на группы ответы и данные для их получения

In [32]:
common_features=list(set(train.columns).intersection(set(test.columns)))

In [34]:
y_train=train['bins']
X_train=train[common_features]
X_test=test[common_features]

Зададим параметры для классификатора и обучим наш искуственный интеллект

In [35]:
%%time
model=xgb.XGBClassifier(
 learning_rate =0.01,
 n_estimators=5000,
 max_depth=4,
 min_child_weight=6,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.005,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
model.fit(X_train,y_train)

Wall time: 3h 33min 48s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=4,
              min_child_weight=6, missing=None, n_estimators=5000, n_jobs=1,
              nthread=4, objective='multi:softprob', random_state=0,
              reg_alpha=0.005, reg_lambda=1, scale_pos_weight=1, seed=27,
              silent=None, subsample=0.8, verbosity=1)

Выведем ответ искусственного интеллекта для тестовых значений

In [36]:
pred=model.predict(X_test)

In [37]:
pred

array([0, 2, 0, ..., 0, 2, 0], dtype=int64)

In [38]:
submission = pd.DataFrame({'bins': pred}, index=test.client_id)
submission.head()

,bins
client_id,
28571,0
27046,2
13240,0
19974,2
10505,1


И скачаем его

In [39]:
import time
import os

current_timestamp = int(time.time())
submission_path = 'submissions/{}.csv'.format(current_timestamp)

if not os.path.exists('submissions'):
    os.makedirs('submissions')

print(submission_path)
submission.to_csv(submission_path, index=True)

submissions/1575149049.csv


На тестовых значениях такое решение предугадало 64,61% 	